# Objectives of this lab

In this lab, we will:
- Walk through how a Chainlink oracle works
- Participate in our own oracle contract that collects values from the class
- Demonstrate resilience against adversarial values

In [ ]:
from web3 import Web3

# Change this to use your own RPC URL for Goerli Testnet
web3 = Web3(Web3.HTTPProvider('YOUR GOERLI RPC URL HERE'))

# Exploring production oracles (Chainlink):

[Chainlink](https://chain.link/) is one of the most popular oracle providers on Ethereum. Let's take a high-level look at how the smart contracts for [the ETH-USD oracle on Chainlink](https://data.chain.link/ethereum/mainnet/crypto-usd/eth-usd) work for an example of how oracles can be implemented.

An end-to-end view of a price feed such as this consists of three types of contracts: 
1. A consumer contract
2. A proxy contract
3. An aggregator contract

### Consumer contract

The consumer contract is any piece of code that queries the aggregator for the oracle values. For the assignments, we would be writing a contract in Solidity. Below is a sample code for doing the same in Python.

In [ ]:
# ETH/USD Chainlink data feed address (Goerli)
ETH_USD_PRICE_FEED_ADDR = '0xD4a33860578De61DBAbDc8BFdb98FD742fA7028e'

# AggregatorV3Interface ABI
abi = '[{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'

# Set up contract instance
contract = web3.eth.contract(address=ETH_USD_PRICE_FEED_ADDR, abi=abi)

# Make call to latestRoundData()
latest_round_data = contract.functions.latestRoundData().call() # returned format is roundId, answer, startedAt, updatedAt, answeredInRound

price = latest_round_data[1] / 10**8 # reported with 8 decimals
print(f"Current oracle reported ETH-USD price: {price}")

### Proxy contract

Proxy contracts are on-chain proxies that point to the aggregator for a particular data feed. Using proxies enables the underlying aggregator to be upgraded without any service interruption to consuming contracts.

The proxy contract for ETH-USD is deployed [here on mainnet](https://etherscan.io/address/0x5f4ec3df9cbd43714fe2740f5e3616155c5b8419#code) and [here on Goerli](https://goerli.etherscan.io/address/0xD4a33860578De61DBAbDc8BFdb98FD742fA7028e).

### Aggregator contract

The contract code for the aggregator is given [here](https://github.com/smartcontractkit/libocr/blob/master/contract/AccessControlledOffchainAggregator.sol). An aggregator is the contract that receives periodic data updates from the network of nodes that have agreed to post their data to a particular Chainlink feed. The data collection and aggregation is done off-chain and the data is then posted on-chain. Aggregators store aggregated data on-chain so that consumers can retrieve it and and act upon it within the same transaction.

You can access this data using the Data Feed address and the [AggregatorV3Interface contract](https://github.com/smartcontractkit/chainlink/blob/develop/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol).

Aggregators receive updates from the oracle network only when the *Deviation Threshold* or *Heartbeat Threshold* triggers an update during an aggregation round. The first condition that is met triggers an update to the data.

- Deviation Threshold: A new aggregation round starts when a node identifies that the off-chain values deviate by more than the defined deviation threshold from the on-chain value. Individual nodes monitor one or more data providers for each feed.
- Heartbeat Threshold: A new aggregation round starts after a specified amount of time from the last update.

# Experimenting with our own oracle contract

The contract at `0xd1942f6B3Ee23918Ff3703cA1c6D76B6dA27ccf5` is an example of a very simple decentralized oracle implementation we created. Note that it is designed purely for an educational example and should not be used in production. There are many gas inefficiencies and obvious potential attack vectors, including a trivial Sybil attack.

The oracle works as follows:
- There are 3 public functions: `readValue()`, `reportValue(uint256)`, `processUpdate()`
- Anyone can call `readValue()` to get the current oracle value and the block number that the value was last updated
- Anyone can call `reportValue(uint256)` to report a value for the next round must pay `REQUIRED_PAYMENT` to do so
- Anyone can call `processUpdate()` after `UPDATE_INTERVAL` blocks has passed since the last update
    - The caller receives 10% of all payments in the round as an incentive
    - The median value of all reported values is chosen as the latest oracle value
    - All accounts that reported the chosen value split the remaining 90% of funds
    
This is a simple example design of an oracle that incentivises people to report the "correct" value, as they will be rewarded for doing so.

Below is the contract for the oracle. You can also view the [verified contract on Etherscan](https://goerli.etherscan.io/address/0xd1942f6b3ee23918ff3703ca1c6d76b6da27ccf5#code).

```javascript
// SPDX-License-Identifier: UNLICENSED

pragma solidity ^0.8.17;

/*
    This contract is an example of a very simple decentralized oracle implementation. Note that it is designed purely
    for an educational example and should not be used in production. There are many gas inefficiencies and obvious
    potential attack vectors, including a trivial Sybil attack.

    The oracle works as follows:
        - The oracle value is updated once every round, where each round is at least UPDATE_INTERVAL blocks long
        - Anyone can call reportValue() to report a value for the next round must pay REQUIRED_PAYMENT to do so
        - Anyone can call processUpdate() after UPDATE_INTERVAL has passed since the last update
            - The caller receives 10% of all payments in the round as an incentive
            - The median value of all reported values is chosen as the oracle value
            - All accounts that reported the chosen value split the remaining 90% of funds
*/
contract Oracle {
    // Used to keep track of who reported what values
    struct ValueReport {
        address reporter;
        uint256 value;
    }

    uint256 public constant REQUIRED_PAYMENT = 0.01 ether; // Require all value reports to include a small payment
    uint256 public constant UPDATE_INTERVAL = 15; // Require at least 15 blocks between updates (roughly 3 minutes)

    ValueReport[] reports; // Keeps track of reports in this round
    uint256 latestValue; // Value from the last round
    uint256 lastUpdateBlock; // Block number of the last value update

    // This is the function other contracts will interact with to get the oracle value
    function readValue() external view returns (uint256, uint256) {
        return (latestValue, lastUpdateBlock);
    }

    // This is the function used to report a value for the current round
    function reportValue(uint256 value) external payable {
        require(
            msg.value == REQUIRED_PAYMENT,
            "Must send REQUIRED_PAYMENT ether"
        );

        // Ensure no one reports multiple values per round
        // Gas-inefficient and vulnerable to Sybil attacks
        for (uint256 i = 0; i < reports.length; i++) {
            require(
                msg.sender != reports[i].reporter,
                "Can only report one value per round"
            );
        }

        reports.push(ValueReport(msg.sender, value));
    }

    // This is the function that determines the oracle value for the current round
    // Value is chosen from the median of all reports in the round
    function processUpdate() external {
        require(
            block.number - lastUpdateBlock >= UPDATE_INTERVAL,
            "Must wait at least UPDATE_INTERVAL blocks since previous update"
        );
        require(reports.length >= 1, "No new reports in this round");

        latestValue = median(reports, reports.length);

        // Send 10% of all funds from this round to the caller (incentive for calling)
        msg.sender.call{value: address(this).balance / 10}("");

        // Count the number of people who reported the median value
        uint256 numCorrect = 0;
        for (uint256 i = 0; i < reports.length; i++) {
            if (reports[i].value == latestValue) {
                numCorrect++;
            }
        }

        // Distribute the remaining funds equally among everyone who was correct (incentive for reporting correct value)
        uint256 reward = address(this).balance / numCorrect;
        for (uint256 i = 0; i < reports.length; i++) {
            if (reports[i].value == latestValue) {
                reports[i].reporter.call{value: reward}("");
            }
        }

        lastUpdateBlock = block.number;
        delete reports;
    }

    // Internal helper function for finding the median
    // Very gas inefficient
    // Adapted from: https://github.com/OpenZeppelin/openzeppelin-contracts/issues/1548#issuecomment-779249419

    function swap(
        ValueReport[] memory array,
        uint256 i,
        uint256 j
    ) internal pure {
        (array[i], array[j]) = (array[j], array[i]);
    }

    function sort(
        ValueReport[] memory array,
        uint256 begin,
        uint256 end
    ) internal pure {
        if (begin < end) {
            uint256 j = begin;
            uint256 pivot = array[j].value;
            for (uint256 i = begin + 1; i < end; ++i) {
                if (array[i].value < pivot) {
                    swap(array, i, ++j);
                }
            }
            swap(array, begin, j);
            sort(array, begin, j);
            sort(array, j + 1, end);
        }
    }

    function median(ValueReport[] memory array, uint256 length)
        internal
        pure
        returns (uint256)
    {
        sort(array, 0, length);
        return array[length / 2].value; // We take the larger value in the even case just to make it simple
    }
}
```

In [ ]:
ORACLE_CONTRACT_ADDRESS = "0xd1942f6B3Ee23918Ff3703cA1c6D76B6dA27ccf5"
ABI = '[{"inputs":[],"name":"REQUIRED_PAYMENT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"UPDATE_INTERVAL","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"processUpdate","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"readValue","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"value","type":"uint256"}],"name":"reportValue","outputs":[],"stateMutability":"payable","type":"function"}]'

# Read the latest oracle value

Let's read the latest value from the oracle.

In [ ]:
contract = web3.eth.contract(address=ORACLE_CONTRACT_ADDRESS, abi=ABI)
value, block_number = contract.functions.readValue().call()

print(f"The current oracle value is {value}")
print(f"Value was last updated at block {block_number} ({web3.eth.block_number - block_number} blocks ago)")

## Reporting a value to the oracle

Now, let's a report a value to the oracle for the upcoming update. 

Remember, everyone who reports the median value during this round will split all the rewards, so it is in your interest to report a value as close as possible to what you think is the correct value. You can only make one report per round.

- Go to the [write contract page](https://goerli.etherscan.io/address/0xd1942f6b3ee23918ff3703ca1c6d76b6da27ccf5#writeContract) for the oracle on Etherscan.
- Make a transaction calling `reportValue()` by providing 0.01 Goerli ETH along with the value you wish to report

Whenever at least `UPDATE_INTERVAL` blocks have passed since the last update, anyone can call `processUpdate()` to trigger the oracle update and rewards distribution. This example contract distributes Goerli ETH as a reward, but you could easily imagine that the reward could be the protocol's token or something else similar instead.

Try being the first to call `processUpdate()` to get the caller reward for free. (HINT: calling `readValue()` also tells you the last block the value was updated at. The variable `UPDATE_INTERVAL` defines the minimum number of blocks between intervals.

## Reporting adversarial values

Oracles should be resilient to malicious input. For this part, we want approximately 1/3 of the class to report an adverserial value and hopefully demonstrate that our oracle still produces a reasonable value.

When instructed to do so:
- If your birth month is January, February, March or April
    - Call `reportValue()` and report an adverserial value
- Otherwise
    - Call `reportValue()` and report what you believe is the true value

After the next update, call `readValue()` to see what value the oracle decided on. Hopefully, it seems reasonable!